In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import  OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import r2_score

import plotly.express as px
import plotly as plt
import plotly.graph_objects as go
import plotly.io as pio

pio.renderers.default = "vscode" # to be replaced by "iframe" if working on JULIE

# IMPORT DATASET

In [2]:
df_original=pd.read_csv('Walmart_Store_sales.csv')

In [4]:
#make a copy to work on it
df=df_original.copy()

In [5]:
df.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,18-02-2011,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,25-03-2011,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,27-07-2012,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,NaN,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,28-05-2010,1644470.66,0.0,78.89,2.759,212.412888,7.092


# Explore dataset

In [6]:
#Basic stats
print("Number of rows : {}".format(df.shape[0]))
print("Number of columns : {}".format(df.shape[1]))
print()

print("Basics statistics: ")
data_desc = df.describe(include='all')
display(data_desc)
print()

print("Percentage of missing values: ")
display(100*df.isnull().sum()/df.shape[0])

Number of rows : 150
Number of columns : 8

Basics statistics: 


,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
count,150.000000,132,1.360000e+02,138.000000,132.000000,136.000000,138.000000,135.000000
unique,NaN,85,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,19-10-2012,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN
mean,9.866667,NaN,1.249536e+06,0.079710,61.398106,3.320853,179.898509,7.598430
std,6.231191,NaN,6.474630e+05,0.271831,18.378901,0.478149,40.274956,1.577173
min,1.000000,NaN,2.689290e+05,0.000000,18.790000,2.514000,126.111903,5.143000
25%,4.000000,NaN,6.050757e+05,0.000000,45.587500,2.852250,131.970831,6.597500
50%,9.000000,NaN,1.261424e+06,0.000000,62.985000,3.451000,197.908893,7.470000
75%,15.750000,NaN,1.806386e+06,0.000000,76.345000,3.706250,214.934616,8.150000



Percentage of missing values: 


Store            0.000000
Date            12.000000
Weekly_Sales     9.333333
Holiday_Flag     8.000000
Temperature     12.000000
Fuel_Price       9.333333
CPI              8.000000
Unemployment    10.000000
dtype: float64

# Study outliers

In [7]:
# We fix as outlier all the values up to the mean + 3 times the standard deviation for each feature
max_Temp=df['Temperature'].mean() + 3*df['Temperature'].std()
max_Fuel=df['Fuel_Price'].mean() + 3*df['Fuel_Price'].std()
max_CPI=df['CPI'].mean() + 3*df['CPI'].std()
max_Unempl=df['Unemployment'].mean() + 3*df['Unemployment'].std()
print('we can consider as outliers, values up to: ')
print(f'Temperature = {max_Temp}, Fuel_Price = {max_Fuel}, CPI = {max_CPI}, Unemployment = {max_Unempl}')

we can consider as outliers, values up to: 
Temperature = 116.53480791969432, Fuel_Price = 4.75530003695527, CPI = 300.7233775900417, Unemployment = 12.329947132356104


In [8]:
# We fix as outlier all the values less than the mean + 3 times the standard deviation for each feature
min_Temp=df['Temperature'].mean() - 3*df['Temperature'].std()
min_Fuel=df['Fuel_Price'].mean() - 3*df['Fuel_Price'].std()
min_CPI=df['CPI'].mean() - 3*df['CPI'].std()
min_Unempl=df['Unemployment'].mean() - 3*df['Unemployment'].std()
print('we can consider as outliers, values less than: ')
print(f'Temperature = {min_Temp}, Fuel_Price = {min_Fuel}, CPI = {min_CPI}, Unemployment = {min_Unempl}')

we can consider as outliers, values less than: 
Temperature = 6.261404201517784, Fuel_Price = 1.8864058453976682, CPI = 59.073639844740796, Unemployment = 2.8669121269031557


In [11]:
# Outliers vizualization
fig = px.box(df, y="Unemployment")
fig.show()

In [10]:
#We check how many rows with null value as target y (Weekly_Sales)
df['Weekly_Sales'].isnull().sum()

14

In [ ]:
# We drop these rows
df=df.dropna(subset = ['Weekly_Sales'])
df.Weekly_Sales.isnull().sum()

In [ ]:
df.shape

In [ ]:
#How many rows I'll drop = 5
(df['Unemployment'] > df['Unemployment'].mean() + 3*df['Unemployment'].std()).value_counts()

In [ ]:
# Dropping outliers in Unemployment feature (using masks) (we keep the null values here)

print('Dropping outliers in Unemployment...')
to_keep = (df['Unemployment'] < (df['Unemployment'].mean() + 3*df['Unemployment'].std())) | (df['Unemployment'].isnull())
df = df.loc[to_keep,:]
print('Done. Number of lines remaining : ', df.shape[0])
print()

In [12]:
#We will split this column by creating column for each: day, year, month.... 
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['DayOfWeek'] = df['Date'].dt.dayofweek
df=df.drop(columns=['Date'])
df.head()

/var/folders/t3/rkks8m514635lhc10vfq7kn80000gn/T/ipykernel_13747/3569156834.py:2: UserWarning:

Parsing '18-02-2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/var/folders/t3/rkks8m514635lhc10vfq7kn80000gn/T/ipykernel_13747/3569156834.py:2: UserWarning:

Parsing '25-03-2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/var/folders/t3/rkks8m514635lhc10vfq7kn80000gn/T/ipykernel_13747/3569156834.py:2: UserWarning:

Parsing '27-07-2012' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/var/folders/t3/rkks8m514635lhc10vfq7kn80000gn/T/ipykernel_13747/3569156834.py:2: UserWarning:

Parsing '28-05-2010' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.

/var/folders/t3/rkks8m514635lhc10vfq7kn80000gn/T/ipykernel_13747/3569156834.py:2: UserWarning:

Parsing '19-08-2011' in DD/MM/YYYY f

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment,Year,Month,Day,DayOfWeek
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858,2011.0,2.0,18.0,4.0
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470,2011.0,3.0,25.0,4.0
2,17.0,NaN,0.0,NaN,NaN,130.719581,5.936,2012.0,7.0,27.0,4.0
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346,NaN,NaN,NaN,NaN
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092,2010.0,5.0,28.0,4.0


In [33]:
#Let's visualize total weekly sales per store
sales=df["Weekly_Sales"].groupby(df["Store"]).sum()
px.bar(sales)

In [16]:
first_df=df.iloc[:,:7]
first_df.head()

,Store,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,6.0,1572117.54,NaN,59.61,3.045,214.777523,6.858
1,13.0,1807545.43,0.0,42.38,3.435,128.616064,7.470
2,17.0,NaN,0.0,NaN,NaN,130.719581,5.936
3,11.0,1244390.03,0.0,84.57,NaN,214.556497,7.346
4,6.0,1644470.66,0.0,78.89,2.759,212.412888,7.092


In [42]:
# Visualize pairwise dependencies
fig = px.scatter_matrix(df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height = 1200, width = 1200)
fig.show()

In [ ]:
# Visualize pairwise dependencies without colums day/month/year/day-of-week
fig = px.scatter_matrix(first_df)
fig.update_layout(
        title = go.layout.Title(text = "Bivariate analysis", x = 0.5), showlegend = False, 
            autosize=False, height = 1200, width = 1200)
fig.show()

In [ ]:
# Univariate analysis
# Distribution of each numeric variable
num_features = ['Weekly_Sales', 'Fuel_Price', 'Temperature', 'CPI', 'Unemployment']
for i in range(len(num_features)):
    fig = px.histogram(df[num_features[i]])
    fig.show()


In [ ]:
sns.barplot(data=df, x="DayOfWeek", y="Weekly_Sales" )

In [ ]:
sns.barplot(data=df, x="Month", y="Weekly_Sales" )

In [ ]:
sns.relplot(x="Temperature", y="Weekly_Sales", data = df, kind="line", height = 5, aspect = 5)
#sns.catplot(data=df, x="Temperature", y="Weekly_Sales", kind="boxen")

In [ ]:
#sns.relplot(x="Store", y="Weekly_Sales", data = df, kind="line", height = 5, aspect = 5)
sns.barplot(data=df, x="Store", y="Weekly_Sales", hue='Holiday_Flag')


In [ ]:
# Correlation matrix
corr_matrix = df.corr().round(2)

import plotly.figure_factory as ff

fig = ff.create_annotated_heatmap(corr_matrix.values,
                                  x = corr_matrix.columns.tolist(),
                                  y = corr_matrix.index.tolist())


fig.show()


In [ ]:
df.columns

In [ ]:
# Separate target variable Y from features X
target_name = 'Weekly_Sales'
features_list = ["Store","Holiday_Flag","Temperature","Fuel_Price","CPI","Unemployment","Year","Month","Day","DayOfWeek"]


X = df.loc[:,features_list]
Y = df.loc[:,target_name]
print("Separating labels from features...")
Y = df.loc[:,target_name]
X = df.drop(target_name, axis = 1) # All columns are kept, except the target
print("...Done.")
print(Y.head())
print()
print(X.head())
print()


In [ ]:
# Divide dataset Train set & Test set 
print("Dividing into train and test sets...")
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=0)
print("...Done.")
print()

In [ ]:
#Categorical variables : Store, Holiday_Flag, DayOfWeek
#Numerical variables : Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day
# Create pipeline for numeric features
numeric_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment'] # Names of numeric columns in X_train/X_test
date_features = ['Year', 'Month','Day', 'DayOfWeek']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),# missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])
numeric_transformer_date = Pipeline(steps=[
    ('imputer', KNNImputer()),# missing values will be replaced by columns' median
    ('scaler', StandardScaler())
])


In [ ]:
# Create pipeline for categorical features
categorical_features = ['Store', 'Holiday_Flag'] # Names of categorical columns in X_train/X_test
categorical_transformer = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')), # missing values will be replaced by most frequent value
    ('encoder', OneHotEncoder(drop='first')) # first column will be dropped to avoid creating correlations between features
    ])

In [ ]:
# Use ColumnTransformer to make a preprocessor object that describes all the treatments to be done
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('date',numeric_transformer_date, date_features),
        ('cat', categorical_transformer, categorical_features)
    ])

In [ ]:
# Preprocessings on train set
print("Performing preprocessings on train set...")
print(X_train.head())
X_train = preprocessor.fit_transform(X_train)
print('...Done.')
print(X_train[0:5]) # MUST use this syntax because X_train is a numpy array and not a pandas DataFrame anymore
print()

# Preprocessings on test set
print("Performing preprocessings on test set...")
print(X_test.head()) 
X_test = preprocessor.transform(X_test) # Don't fit again !! The test set is used for validating decisions
# we made based on the training set, therefore we can only apply transformations that were parametered using the training set.
# Otherwise this creates what is called a leak from the test set which will introduce a bias in all your results.
print('...Done.')
print(X_test[0:5,:]) # MUST use this syntax because X_test is a numpy array and not a pandas DataFrame anymore
print()


In [ ]:
#Baseline Model: Linear Regression =>Train model
print("Train model...")
regressor = LinearRegression()
regressor.fit(X_train, Y_train)
print("...Done.")

In [ ]:
# Predictions on training set
print("Predictions on training set...")
Y_train_pred = regressor.predict(X_train)
print("...Done.")
print(Y_train_pred)
print()

In [ ]:
# Predictions on test set
print("Predictions on test set...")
Y_test_pred = regressor.predict(X_test)
print("...Done.")
print(Y_test_pred)
print()

In [ ]:
# Print R^2 scores
print("R2 score on training set : ", r2_score(Y_train, Y_train_pred))
print("R2 score on test set : ", r2_score(Y_test, Y_test_pred))

In [ ]:
#Nous obtenons un R2 supérieur sur le Train, nous sommes peut etre en overfitting
print("Allons vérifier si cette différence entre le score train et test est significative")

In [ ]:
#Cross Validation
print("3-fold cross-validation...")
scores = cross_val_score(regressor, X_train, Y_train, cv=5)
print('The cross-validated R2-score is : ', scores.mean())
print('The standard deviation is : ', scores.std())


In [ ]:
print('Nous avons obtenu un score R2 test de: {} or ce score + notre écart type sur les différents scores obtenus avec notre Cross Validation reste inférieur à notre score R2 train de : {}'.format((round(r2_score(Y_test, Y_test_pred),5)),(round(r2_score(Y_train, Y_train_pred),5))))
print('Nous sommes donc bel et bien en overfitting')

In [ ]:
regressor.coef_

In [ ]:
column_names = []
for name, pipeline, features_list in preprocessor.transformers_: # loop over pipelines
    if name == 'num': # if pipeline is for numeric variables
        features = features_list # just get the names of columns to which it has been applied
    else: # if pipeline is for categorical variables
        features = pipeline.named_steps['encoder'].get_feature_names_out() # get output columns names from OneHotEncoder
    column_names.extend(features) # concatenate features names
        
print("Names of columns corresponding to each coefficient: ", column_names)


In [ ]:
# Create a pandas DataFrame
coefs_Lin = pd.DataFrame(index = column_names, data = regressor.coef_.transpose(), columns=["coefficients"])
coefs_Lin

In [ ]:
# Compute abs() and sort values
feature_importance = abs(coefs_Lin).sort_values(by = 'coefficients')
feature_importance

In [ ]:
fig = px.bar(feature_importance)
fig.show()

In [ ]:
#Part 3 : Fight overfitting=>Ridge
Ridge1 = Ridge()

Ridge1.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", Ridge1.score(X_train, Y_train))
print("R2 score on test set : ", Ridge1.score(X_test, Y_test))


In [ ]:
#Part 3 : Fight overfitting=>Ridge
Ridge2 = Ridge(alpha = 5)

Ridge2.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", Ridge2.score(X_train, Y_train))
print("R2 score on test set : ", Ridge2.score(X_test, Y_test))

In [ ]:
#Part 3 : Fight overfitting =>Lasso
Lasso1 = Lasso()

Lasso1.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", Lasso1.score(X_train, Y_train))
print("R2 score on test set : ", Lasso1.score(X_test, Y_test))


In [ ]:
#Part 3 : Fight overfitting =>Lasso
Lasso2 = Lasso(alpha=200)

Lasso2.fit(X_train, Y_train)
# Print R^2 scores
print("R2 score on training set : ", Lasso2.score(X_train, Y_train))
print("R2 score on test set : ", Lasso2.score(X_test, Y_test))

In [ ]:
# Perform grid search with Ridge
print("Grid search...")
regressorR = Ridge()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0, 2.0,2.5, 3.0, 4.0] # 0 corresponds to no regularization
}
gridsearchR = GridSearchCV(regressorR, param_grid = params) # cv : the number of folds to be used for CV
gridsearchR.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearchR.best_params_)
print("Best R2 score : ", gridsearchR.best_score_)


In [ ]:
# Perform grid search with Ridge
print("Grid search...")
regressorL = Lasso()
# Grid of values to be tested
params = {
    'alpha': [0.0, 0.1, 0.5, 1.0, 2.0,2.5] # 0 corresponds to no regularization
}
gridsearchL = GridSearchCV(regressorL, param_grid = params) # cv : the number of folds to be used for CV
gridsearchL.fit(X_train, Y_train)
print("...Done.")
print("Best hyperparameters : ", gridsearchL.best_params_)
print("Best R2 score : ", gridsearchL.best_score_)